In [1]:
import stanza
import pandas as pd
import numpy as np
import re
import tensorflow as tf
import tqdm as notebook_tqdm

In [2]:
nlp = stanza.Pipeline(lang="en",use_gpu=True)

2024-01-11 21:48:29 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-01-11 21:48:30 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| constituency | wsj       |
| depparse     | combined  |
| sentiment    | sstplus   |
| ner          | ontonotes |

2024-01-11 21:48:30 WARNING: GPU requested, but is not available!
2024-01-11 21:48:30 INFO: Using device: cpu
2024-01-11 21:48:30 INFO: Loading: tokenize
2024-01-11 21:48:31 INFO: Loading: pos
2024-01-11 21:48:31 INFO: Loading: lemma
2024-01-11 21:48:31 INFO: Loading: constituency
2024-01-11 21:48:31 INFO: Loading: depparse
2024-01-11 21:48:32 INFO: Loading: sentiment
2024-01-11 21:48:32 INFO: Loading: ner
2024-01-11 21:48:33 INFO: Done loading processors!


In [3]:
doc = nlp("An elderly man sitting in a small shop")
print("token:")
doc.sentences[0].print_tokens()

token:
<Token id=1;words=[<Word id=1;text=An;lemma=a;upos=DET;xpos=DT;feats=Definite=Ind|PronType=Art;head=3;deprel=det>]>
<Token id=2;words=[<Word id=2;text=elderly;lemma=elderly;upos=ADJ;xpos=JJ;feats=Degree=Pos;head=3;deprel=amod>]>
<Token id=3;words=[<Word id=3;text=man;lemma=man;upos=NOUN;xpos=NN;feats=Number=Sing;head=0;deprel=root>]>
<Token id=4;words=[<Word id=4;text=sitting;lemma=sit;upos=VERB;xpos=VBG;feats=VerbForm=Ger;head=3;deprel=acl>]>
<Token id=5;words=[<Word id=5;text=in;lemma=in;upos=ADP;xpos=IN;head=8;deprel=case>]>
<Token id=6;words=[<Word id=6;text=a;lemma=a;upos=DET;xpos=DT;feats=Definite=Ind|PronType=Art;head=8;deprel=det>]>
<Token id=7;words=[<Word id=7;text=small;lemma=small;upos=ADJ;xpos=JJ;feats=Degree=Pos;head=8;deprel=amod>]>
<Token id=8;words=[<Word id=8;text=shop;lemma=shop;upos=NOUN;xpos=NN;feats=Number=Sing;head=4;deprel=obl>]>


In [4]:
# Import dataframe
import pandas as pd
train_file = 'D:\\Suwija Putra\\datasetTextualEntailment\\snli_1.0\\snli_1.0_train.jsonl'
df_train = pd.read_json(train_file, lines=True) 
df_train

,annotator_labels,captionID,gold_label,pairID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse
0,[neutral],3416050480.jpg#4,neutral,3416050480.jpg#4r1n,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,A person is training his horse for a competition.,( ( A person ) ( ( is ( ( training ( his horse...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...
1,[contradiction],3416050480.jpg#4,contradiction,3416050480.jpg#4r1c,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,"A person is at a diner, ordering an omelette.",( ( A person ) ( ( ( ( is ( at ( a diner ) ) )...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...
2,[entailment],3416050480.jpg#4,entailment,3416050480.jpg#4r1e,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,"A person is outdoors, on a horse.","( ( A person ) ( ( ( ( is outdoors ) , ) ( on ...",(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...
3,[neutral],2267923837.jpg#2,neutral,2267923837.jpg#2r1n,Children smiling and waving at camera,( Children ( ( ( smiling and ) waving ) ( at c...,(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,They are smiling at their parents,( They ( are ( smiling ( at ( their parents ) ...,(ROOT (S (NP (PRP They)) (VP (VBP are) (VP (VB...
4,[entailment],2267923837.jpg#2,entailment,2267923837.jpg#2r1e,Children smiling and waving at camera,( Children ( ( ( smiling and ) waving ) ( at c...,(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,There are children present,( There ( ( are children ) present ) ),(ROOT (S (NP (EX There)) (VP (VBP are) (NP (NN...
...,...,...,...,...,...,...,...,...,...,...
550147,[contradiction],2267923837.jpg#3,contradiction,2267923837.jpg#3r1c,Four dirty and barefooted children.,( ( ( ( Four dirty ) and ) ( barefooted childr...,(ROOT (NP (NP (CD Four) (NNS dirty)) (CC and) ...,four kids won awards for 'cleanest feet',( ( four kids ) ( ( won awards ) ( ( ( for ` )...,(ROOT (S (NP (CD four) (NNS kids)) (VP (VBD wo...
550148,[neutral],2267923837.jpg#3,neutral,2267923837.jpg#3r1n,Four dirty and barefooted children.,( ( ( ( Four dirty ) and ) ( barefooted childr...,(ROOT (NP (NP (CD Four) (NNS dirty)) (CC and) ...,"four homeless children had their shoes stolen,...",( ( ( ( ( ( four ( homeless children ) ) ( had...,(ROOT (S (S (NP (CD four) (JJ homeless) (NNS c...
550149,[neutral],7979219683.jpg#2,neutral,7979219683.jpg#2r1n,A man is surfing in a bodysuit in beautiful bl...,( ( A man ) ( ( is ( surfing ( in ( ( a bodysu...,(ROOT (S (NP (DT A) (NN man)) (VP (VBZ is) (VP...,A man in a bodysuit is competing in a surfing ...,( ( ( A man ) ( in ( a bodysuit ) ) ) ( ( is (...,(ROOT (S (NP (NP (DT A) (NN man)) (PP (IN in) ...
550150,[contradiction],7979219683.jpg#2,contradiction,7979219683.jpg#2r1c,A man is surfing in a bodysuit in beautiful bl...,( ( A man ) ( ( is ( surfing ( in ( ( a bodysu...,(ROOT (S (NP (DT A) (NN man)) (VP (VBZ is) (VP...,A man in a business suit is heading to a board...,( ( ( A man ) ( in ( a ( business suit ) ) ) )...,(ROOT (S (NP (NP (DT A) (NN man)) (PP (IN in) ...


In [5]:
df_train.shape

(550152, 10)

In [6]:
import string
df_train_kalimat1 = df_train
df_train_kalimat1 = df_train_kalimat1['sentence1']
# Ambil 10 data pertama
df_train_kalimat2 = df_train
# Ambil kalimat kedua saja sebagai contoh
df_train_kalimat2 = df_train_kalimat2['sentence2']

df_label = df_train
df_label = df_label['gold_label']
# Ubah ke lowercase
def remove_punctuation(text):
    text = text.lower()
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)
df_train_kalimat1 = df_train_kalimat1.apply(remove_punctuation)
df_train_kalimat2 = df_train_kalimat2.apply(remove_punctuation)

In [7]:
list_dependencies_2 = list()
for i in df_train_kalimat2:
    # print(i)
    doc = nlp(i)
    dependencies_list = doc.sentences[0].dependencies
    list_dependencies = list()
    for x in range(len(dependencies_list)):
        # print(dependencies_list[x][0].text)
        # print(dependencies_list[x][2].text)
        list_dependencies.append([dependencies_list[x][0].text,dependencies_list[x][2].text])
    # print(list_dependencies)
    list_dependencies_2.append(list_dependencies)
# list_dependencies_all

list_dependencies_1 = list()
for i in df_train_kalimat1:
    # print(i)
    doc = nlp(i)
    dependencies_list = doc.sentences[0].dependencies
    list_dependencies = list()
    for x in range(len(dependencies_list)):
        # print(dependencies_list[x][0].text)
        # print(dependencies_list[x][2].text)
        list_dependencies.append([dependencies_list[x][0].text,dependencies_list[x][2].text])
    # print(list_dependencies)
    list_dependencies_1.append(list_dependencies)

In [8]:
filtered_list1 = [[sublist for sublist in subsublist if 'ROOT' not in sublist] for subsublist in list_dependencies_1]
filtered_list2 = [[sublist for sublist in subsublist if 'ROOT' not in sublist] for subsublist in list_dependencies_2]
print(df_train_kalimat1[0])
print(filtered_list1[0])

a person on a horse jumps over a broken down airplane
[['person', 'a'], ['jumps', 'person'], ['horse', 'on'], ['horse', 'a'], ['person', 'horse'], ['airplane', 'over'], ['airplane', 'a'], ['airplane', 'broken'], ['broken', 'down'], ['jumps', 'airplane']]


In [9]:
# Head
head_1 = list()
for item in filtered_list1:
    head_item = list()
    for i in item:
        head_item.append(i[0])
    head_1.append(head_item)
# print(head_1)

dependecy_1 = list()
for item in filtered_list1:
    dependecy_item = list()
    for i in item:
        dependecy_item.append(i[1])
    dependecy_1.append(dependecy_item)
# print(dependecy_1)

head_2 = list()
for item in filtered_list2:
    head_item = list()
    for i in item:
        head_item.append(i[0])
    head_2.append(head_item)
# print(head_1)

dependecy_2 = list()
for item in filtered_list2:
    dependecy_item = list()
    for i in item:
        dependecy_item.append(i[1])
    dependecy_2.append(dependecy_item)

In [11]:
head_dependecy = pd.DataFrame({'head1':head_1,'dependecy1':dependecy_1,'head2':head_2,'dependecy2':dependecy_2})
head_dependecy = pd.concat([head_dependecy,df_label],axis=1)
head_dependecy

,head1,dependecy1,head2,dependecy2,gold_label
0,"[person, jumps, horse, horse, person, airplane...","[a, person, on, a, horse, over, a, broken, dow...","[person, training, training, horse, training, ...","[a, person, is, his, horse, for, a, competition]",neutral
1,"[person, jumps, horse, horse, person, airplane...","[a, person, on, a, horse, over, a, broken, dow...","[person, diner, diner, diner, diner, diner, om...","[a, person, is, at, a, ordering, an, omelette]",contradiction
2,"[person, jumps, horse, horse, person, airplane...","[a, person, on, a, horse, over, a, broken, dow...","[person, outdoors, outdoors, horse, horse, out...","[a, person, is, on, a, horse]",entailment
3,"[smiling, waving, smiling, camera, waving]","[children, and, waving, at, camera]","[smiling, smiling, parents, parents, smiling]","[they, are, at, their, parents]",neutral
4,"[smiling, waving, smiling, camera, waving]","[children, and, waving, at, camera]","[are, are, are]","[there, children, present]",entailment
...,...,...,...,...,...
550147,"[children, children, barefooted, dirty]","[four, dirty, and, barefooted]","[kids, won, won, feet, feet, won]","[four, kids, awards, for, cleanest, feet]",contradiction
550148,"[children, children, barefooted, dirty]","[four, dirty, and, barefooted]","[children, children, had, shoes, had, had, dir...","[four, homeless, children, their, shoes, stole...",neutral
550149,"[man, surfing, surfing, bodysuit, bodysuit, su...","[a, man, is, in, a, bodysuit, in, beautiful, b...","[man, competing, bodysuit, bodysuit, man, comp...","[a, man, in, a, bodysuit, is, in, a, surfing, ...",neutral
550150,"[man, surfing, surfing, bodysuit, bodysuit, su...","[a, man, is, in, a, bodysuit, in, beautiful, b...","[man, heading, suit, suit, suit, man, heading,...","[a, man, in, a, business, suit, is, to, a, boa...",contradiction


In [ ]:
head_dependecy = pd.DataFrame({'head1':head_1,'dependecy1':dependecy_1,'head2':head_2,'dependecy2':dependecy_2})
head_dependecy = pd.concat([head_dependecy,df_label],axis=1)
head_dependecy

,head1,dependecy1,head2,dependecy2,gold_label
0,"[person, jumps, horse, horse, person, airplane...","[a, person, on, a, horse, over, a, broken, dow...","[person, training, training, horse, training, ...","[a, person, is, his, horse, for, a, competition]",neutral
1,"[person, jumps, horse, horse, person, airplane...","[a, person, on, a, horse, over, a, broken, dow...","[person, diner, diner, diner, diner, diner, om...","[a, person, is, at, a, ordering, an, omelette]",contradiction
2,"[person, jumps, horse, horse, person, airplane...","[a, person, on, a, horse, over, a, broken, dow...","[person, outdoors, outdoors, horse, horse, out...","[a, person, is, on, a, horse]",entailment
3,"[smiling, waving, smiling, camera, waving]","[children, and, waving, at, camera]","[smiling, smiling, parents, parents, smiling]","[they, are, at, their, parents]",neutral
4,"[smiling, waving, smiling, camera, waving]","[children, and, waving, at, camera]","[are, are, are]","[there, children, present]",entailment
...,...,...,...,...,...
550147,"[children, children, barefooted, dirty]","[four, dirty, and, barefooted]","[kids, won, won, feet, feet, won]","[four, kids, awards, for, cleanest, feet]",contradiction
550148,"[children, children, barefooted, dirty]","[four, dirty, and, barefooted]","[children, children, had, shoes, had, had, dir...","[four, homeless, children, their, shoes, stole...",neutral
550149,"[man, surfing, surfing, bodysuit, bodysuit, su...","[a, man, is, in, a, bodysuit, in, beautiful, b...","[man, competing, bodysuit, bodysuit, man, comp...","[a, man, in, a, bodysuit, is, in, a, surfing, ...",neutral
550150,"[man, surfing, surfing, bodysuit, bodysuit, su...","[a, man, is, in, a, bodysuit, in, beautiful, b...","[man, heading, suit, suit, suit, man, heading,...","[a, man, in, a, business, suit, is, to, a, boa...",contradiction


In [12]:
head_dependecy['head1'] = head_dependecy['head1'].apply(lambda x: ' '.join(x))
head_dependecy['dependecy1'] = head_dependecy['dependecy1'].apply(lambda x: ' '.join(x))
head_dependecy['head2'] = head_dependecy['head2'].apply(lambda x: ' '.join(x))
head_dependecy['dependecy2'] = head_dependecy['dependecy2'].apply(lambda x: ' '.join(x))
pd.set_option('display.max_colwidth',1)
head_dependecy

,head1,dependecy1,head2,dependecy2,gold_label
0,person jumps horse horse person airplane airplane airplane broken jumps,a person on a horse over a broken down airplane,person training training horse training competition competition training,a person is his horse for a competition,neutral
1,person jumps horse horse person airplane airplane airplane broken jumps,a person on a horse over a broken down airplane,person diner diner diner diner diner omelette ordering,a person is at a ordering an omelette,contradiction
2,person jumps horse horse person airplane airplane airplane broken jumps,a person on a horse over a broken down airplane,person outdoors outdoors horse horse outdoors,a person is on a horse,entailment
3,smiling waving smiling camera waving,children and waving at camera,smiling smiling parents parents smiling,they are at their parents,neutral
4,smiling waving smiling camera waving,children and waving at camera,are are are,there children present,entailment
...,...,...,...,...,...
550147,children children barefooted dirty,four dirty and barefooted,kids won won feet feet won,four kids awards for cleanest feet,contradiction
550148,children children barefooted dirty,four dirty and barefooted,children children had shoes had had dirty feet dirty dirty had,four homeless children their shoes stolen so their feet are dirty,neutral
550149,man surfing surfing bodysuit bodysuit surfing water water water bodysuit,a man is in a bodysuit in beautiful blue water,man competing bodysuit bodysuit man competing competition competition competition competing,a man in a bodysuit is in a surfing competition,neutral
550150,man surfing surfing bodysuit bodysuit surfing water water water bodysuit,a man is in a bodysuit in beautiful blue water,man heading suit suit suit man heading meeting meeting meeting heading,a man in a business suit is to a board meeting,contradiction


In [13]:
# save the file
head_dependecy.to_csv('D:\\Suwija Putra\\datasetTextualEntailment\\indonli\\dependency_train_snli.csv', index=False) # change the name of the file